### This script shows how to use the existing code in opengrid to create a baseload electricity consumption benchmark. 

In [ ]:
import os, sys
import inspect
import numpy as np


import datetime as dt
import time
import pytz
import pandas as pd
import pdb

script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# add the path to opengrid to sys.path
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))
from opengrid.library import houseprint
from opengrid.library import fluksoapi
from opengrid.library import config
c=config.Config()
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet

if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter, AutoDateLocator

# find tmpo
sys.path.append(c.get('tmpo', 'folder'))
import tmpo

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

import charts

### Script settings


In [ ]:
number_of_days = 7

### We create one big dataframe, the columns are the sensors

In [ ]:
hp = houseprint.load_houseprint_from_file('hp_anonymous.pkl')

In [ ]:
start = pd.Timestamp(time.time() - number_of_days*86400, unit='s')

In [ ]:
tmpos = tmpo.Session()
sensors = hp.get_sensors()
#sensors.remove('b325dbc1a0d62c99a50609e919b9ea06')

In [ ]:
for sensor in sensors:
    s = tmpos.series(sid=sensor, head=start)
    try:    
        s = s.resample(rule='60s', how='max')
        s = s.interpolate(method='time')
        s = s.diff()*3600/60

        # plot with charts (don't show it) and save html

        charts.plot(pd.DataFrame(s), stock=True, 
                    save=os.path.join(c.get('data', 'folder'), 'figures', 'TimeSeries_'+sensor+'.html'), show=False)
    except:
        pass

In [ ]:
len(sensors)